In [ ]:
!pip install datasets -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.8 MB/s eta 0:

In [ ]:
from datasets import load_dataset

dataset = load_dataset("pacovaldez/stackoverflow-questions", split='validation')
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/pacovaldez___parquet/pacovaldez--stackoverflow-questions-e77b48110dd75272/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Dataset({
    features: ['title', 'body', 'label'],
    num_rows: 785098
})

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
texts = dataset['title']
texts[-10:]

['Stop a swing Timer from inside the Action Listener',
 'ASP.net c# - Could not load type',
 'Increase page numbers in LaTeX Beamer',
 'Jquery: Binding change to multiple checkboxes',
 'Why is not equal represented by ~= in Matlab?',
 'Convert PowerShell object to string',
 'Python Raspberry PI - Display USB Serial Data on LCD',
 'How do I download NLTK data?',
 'React native - Data fetch from Async storage before rendering the app component',
 'run multiple steps in Jenkins pipeline']

In [ ]:
embeddings = model.encode(texts[-10000:])

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters = 5)
km.fit(embeddings)

cluster_labels = km.labels_
print(cluster_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[0 0 4 ... 3 2 0]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cs

def search(query, texts, embeddings):
    q_enc = model.encode([query])[0]
    sim_scores = cs([q_enc], embeddings)[0]
    results = [(texts[i], sim_scores[i]) for i in range(len(texts))]
    results.sort(key = lambda x: x[1], reverse = True)
    return results

In [ ]:
query = "Enter an error description: "
s_results = search(query, texts[-700:], embeddings)


import pandas as pd

scores = []
titles = []


for i in range(len(s_results)):
    scores.append(s_results[i][1])
    titles.append(s_results[i][0])

df = pd.DataFrame({"Title": titles, "Scores": scores})
df.head()

,Title,Scores
0,Displaying Two Or More Unordered Lists Next To...,0.447734
1,How to increase font size of contour labels?,0.445150
2,How to unload activity in android,0.442012
3,How do I enable multiple client ids in a Sprin...,0.402297
4,"TypeError ""null has no properties"" when adding...",0.401774


In [ ]:
from tqdm import tqdm
def deploy(query):
    s_results = search(query, texts[-10000:], embeddings)

    scores = []
    titles = []


    for i in range(len(tqdm(s_results))):
        scores.append(s_results[i][1])
        titles.append(s_results[i][0])

    df = pd.DataFrame({"Title": titles, "Scores": scores})
    print(df.head(10))

In [ ]:
deploy('My computer is dead')

  0%|          | 0/10000 [00:00<?, ?it/s]

                                               Title    Scores
0  Intel MPI - mpiexec.exe does not run, only giv...  0.382246
1      Clone of an old bare repository appears empty  0.367142
2                        SC StartService FAILED 1058  0.344036
3  10 GIT lines erased 8Gb form my pc - How to ge...  0.338038
4                  Device keeps disappearing in DDMS  0.336206
5                             Unable to determine OS  0.334231
6            Delete disk from Azure after VM deleted  0.326867
7  NVMe SSD disk on AWS Sagemaker Notebook Instances  0.326655
8  How clear and invalidate ARM v7 processor cach...  0.319613
9          Webstorm crashes and all my settings gone  0.315191
